In [1]:
import numpy as np
import os
import sys

In [2]:
if(os.path.exists("./tabulated/2D_512x256/GamowTable.npy")):
    os.remove('./tabulated/2D_512x256/GamowTable.npy')
    os.remove('./tabulated/2D_512x256/tabLimits.npy')

In [3]:
mainpath,filename = os.path.split(os.path.dirname(os.getcwd()))

emissionpath,mainfolder = os.path.split(mainpath)
emissionpath,mainfolder = os.path.split(emissionpath)

pythonpath = emissionpath + '/mikael/GETELEC/interfaces/web_interface'
sys.path.append(pythonpath)

In [4]:
pythonpath

'/home/mikael/GETELEC/interfaces/web_interface'

In [5]:
from getelec_online import current_metal_emitter, heat_metal_emitter, spectrum_metal_emitter
from getelec_online import current_metal_emitter_threaded

tabulation files not found in the path tabulated/2D_512x256
Producing tabulator by running:
python3 /home/mikael/GETELEC/src/../oldGETELEC/python/tabulateGamow.py -nf 512 -nr 256 -ng 1 -np 4 -nG 128 -o tabulated/2D_512x256
/home/mikael/GETELEC/oldGETELEC/lib/libgetelec.so
 GETELEC: Parameters input file  not found. Default values used.
Running table generation script with tablesize $512 ...


In [6]:
if(os.path.exists("./tabulated/2D_512x256/GamowTable.npy")):
    os.remove('./tabulated/2D_512x256/GamowTable.npy')
    os.remove('./tabulated/2D_512x256/tabLimits.npy')

In [ ]:
def tabulateGamowTable_worker(self, input):

    i, j, k = input
    field = 1 / self.inverseFieldValues[k]
    radius = 1 / self.inverseRadiusValues[j]
    gamma = 1 / self.inverseGammaValues[i]
    
    minimumBarrierDepth = np.array([1.])
    maximumBarrierDepth = np.copy(minimumBarrierDepth)
    G = np.zeros(self.numberOfGammowValues)
    
    getelec.export_gamow_for_energy_range(
        ct.c_double(field), ct.c_double(radius), ct.c_double(gamma), ct.c_int(self.numberOfGammowValues), \
        ct.c_void_p(minimumBarrierDepth.ctypes.data), ct.c_void_p(maximumBarrierDepth.ctypes.data), ct.c_void_p(G.ctypes.data)
    )
    
    barrierDepths = np.linspace(minimumBarrierDepth[0], maximumBarrierDepth[0], self.numberOfGammowValues)
    try:
        fittedPolynomialCoefficients = np.polyfit(barrierDepths, G, self.numberOfPolynomialTerms - 1)
    except(np.RankWarning):
        print("Rank Warning for F = %g, R = %g, gamma = %g" % (field, radius, gamma))
        plt.plot(barrierDepths, G)
        plt.show()
    self.gamowTable[i, j, k, :] = np.append(fittedPolynomialCoefficients, [barrierDepths[0], barrierDepths[-1]])    

def tabulateGamowTable_threaded(self, outputFolder = 'tabulated', threads = 8):
    """Looks for the files where the precaculate barriers are stored. Then it uses interpolation methods to make the most accurate barrier for the given 
    input (electric field, tip radius and gamma exponent). Gtab is stores the polinomial that gives its shape to the barrier.
    """ 

    self.gamowTable = np.ones([self.numberOfGammaValues, self.numberOfRadiusValues, self.numberOfFieldValues, self.numberOfPolynomialTerms + 2])

    with Pool(processes = threads) as pool:
        tasks = (self, [(i, j, k) for i in range(self.numberOfGammaValues) for j in range(self.numberOfRadiusValues) for k in range(self.numberOfFieldValues)])
        pool.map(self.tabulateGamowTable_worker, tasks)

    if (self.numberOfGammaValues == 1):
        self.gamowTable = np.reshape(self.gamowTable, (self.numberOfRadiusValues, self.numberOfFieldValues, self.numberOfPolynomialTerms + 2))
        if (self.numberOfRadiusValues == 1):
            self.gamowTable = np.reshape(self.gamowTable, (self.numberOfFieldValues, self.numberOfPolynomialTerms + 2))
    print(self.gamowTable)

